# 🚀 Projeto Transfer Learning - Cats vs Dogs
Este notebook aplica Transfer Learning usando MobileNetV2 pré-treinada para classificar gatos e cachorros.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

## Carregar o dataset cats_vs_dogs

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

## Pré-processamento

In [ ]:
IMG_SIZE = 160
BATCH_SIZE = 32

def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

train_batches = ds_train.map(format_example).shuffle(1000).batch(BATCH_SIZE)
test_batches = ds_test.map(format_example).batch(BATCH_SIZE)

## Modelo com Transfer Learning

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Treinamento inicial

In [ ]:
history = model.fit(train_batches,
                    epochs=5,
                    validation_data=test_batches)

## Avaliação e gráficos

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(acc, label='Accuracy')
plt.plot(val_acc, label='Val Accuracy')
plt.legend()
plt.show()

## Fine-tuning

In [ ]:
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(train_batches,
                         epochs=5,
                         validation_data=test_batches)